<a href="https://colab.research.google.com/github/Suleymanabdy/Data-Science-Checkpoints./blob/main/DeepFaceMatch_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
# Install OpenCV
!pip install opencv-python

# Install Numpy
!pip install numpy

# Install Keras
!pip install keras

# Install FaceNet via keras-facenet
!pip install keras-facenet

# Install scikit-learn for the SVM classifier
!pip install scikit-learn

# Install Streamlit
!pip install streamlit


In [4]:
!unzip /content/drive/MyDrive/Dataset.zip

Archive:  /content/drive/MyDrive/Dataset.zip
   creating: Dataset/
   creating: Dataset/Test/
   creating: Dataset/Test/cristiano/
  inflating: Dataset/Test/cristiano/0x0.jpg  
  inflating: Dataset/Test/cristiano/210821084705-cristiano-ronaldo-08-14-2021.jpg  
  inflating: Dataset/Test/cristiano/2306163-48022050-2560-1440.jpg  
  inflating: Dataset/Test/cristiano/bc25230a6792d2d2f05092d9ccb7614b330172b3.jpg  
  inflating: Dataset/Test/cristiano/Cristiano-Ronaldo-Manchester-United.jpg  
  inflating: Dataset/Test/cristiano/hi-res-d103d3731d10450b1d7cc4ca84b12c8c_crop_exact.jpg  
  inflating: Dataset/Test/cristiano/Legends-Profile_Cristiano-Ronaldo1523460877263.jpg  
  inflating: Dataset/Test/cristiano/newFile-5.jpg  
  inflating: Dataset/Test/cristiano/skysports-cristiano-ronaldo-real-madrid-football_4178227.jpg  
  inflating: Dataset/Test/cristiano/skysports-cristiano-ronaldo_4510781.jpg  
   creating: Dataset/Test/k.mbappe/
  inflating: Dataset/Test/k.mbappe/k.mbappe-___352884131995629

In [56]:
%%writefile app.py
import os
import cv2
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
from keras_facenet import FaceNet
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import streamlit as st

# **1. Load HAAR Cascade for Face Detection**
def extract_face(image_path, output_size=(160, 160)):
    face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')

    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        face = img[y:y+h, x:x+w]
        face_resized = cv2.resize(face, output_size)
        return face_resized
    return None

Writing app.py


In [57]:
#2.Load Faces from Directory
def load_faces(directory, output_size=(160, 160)):
    faces = []
    for filename in os.listdir(directory):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(directory, filename)
            face = extract_face(image_path, output_size)
            if face is not None:
                faces.append(face)
    return faces

In [58]:
#3.Load Dataset from Parent Directory
def load_dataset(parent_directory, output_size=(160, 160)):
    dataset = []
    labels = []
    for label, person_dir in enumerate(os.listdir(parent_directory)):
        person_dir_path = os.path.join(parent_directory, person_dir)
        if os.path.isdir(person_dir_path):
            faces = load_faces(person_dir_path, output_size)
            dataset.extend(faces)
            labels.extend([label] * len(faces))
    return dataset, labels

In [59]:
#5.Get FaceNet Embedding Function
def get_embedding(model, face_image):
    face_image = np.expand_dims(face_image, axis=0)
    embeddings = model.embeddings(face_image)
    return embeddings

In [60]:
#6.Get VGG16 Embedding Function
def get_vgg16_embedding(face_image):
    model = VGG16(weights='imagenet', include_top=False, input_shape=(160, 160, 3))
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)

    face_image = np.expand_dims(face_image, axis=0)
    face_image = preprocess_input(face_image)

    embedding = model.predict(face_image)
    return embedding

In [61]:
#7.Load Train and Test Datasets
train_directory = '/content/Dataset/Train'
test_directory = '/content/Dataset/Test'

train_faces, train_labels = load_dataset(train_directory)
test_faces, test_labels = load_dataset(test_directory)

In [46]:
#8.Generate Embeddings for Train and Test Faces
#Use FaceNet or VGG16 here
model = FaceNet()  # Use FaceNet model, or switch to VGG16 embedding
train_embeddings = [get_embedding(model, face) for face in train_faces]
test_embeddings = [get_embedding(model, face) for face in test_faces]

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 

In [47]:
#9.Save Generated Embeddings
np.savez('train_embeddings.npz', embeddings=train_embeddings, labels=train_labels)
np.savez('test_embeddings.npz', embeddings=test_embeddings, labels=test_labels)

In [48]:
#10.Train Classifier (SVM in this case)
#Flatten the embeddings to pass to the classifier
train_embeddings_flat = [emb.flatten() for emb in train_embeddings]
test_embeddings_flat = [emb.flatten() for emb in test_embeddings]

In [49]:
# Train an SVM classifier
clf = SVC(kernel='linear', probability=True)
clf.fit(train_embeddings_flat, train_labels)

SVC(kernel='linear', probability=True)

In [50]:
#Predict and evaluate the model
test_predictions = clf.predict(test_embeddings_flat)
print(classification_report(test_labels, test_predictions))

              precision    recall  f1-score   support

           0       0.29      0.40      0.33        10
           1       0.15      0.22      0.18         9
           2       0.00      0.00      0.00         7
           3       0.67      0.40      0.50        10
           4       1.00      0.60      0.75        10

    accuracy                           0.35        46
   macro avg       0.42      0.32      0.35        46
weighted avg       0.45      0.35      0.38        46

